In [1]:
import scipy.io
import pandas as pd 
import numpy as np
import re

In [2]:
import pickle

In [3]:
from datetime import datetime

In [4]:
data_df = pd.read_csv('/home/jujun/temp_credit/dataprosper-loans_0128.csv')

In [5]:
data_df.head()

,Unnamed: 0,Lender_ID,Borrower_ID,Timestamp,Amount,Status,Lender_Rate,Borrower_Rate,Grade
0,0,20504,79275,1182816000,100.0,Charge-off,0.1750,0.1950,B
1,1,6182,59872,1213142400,90.0,Paid,0.1355,0.1355,B
2,2,10118,67702,1155600000,1000.0,Defaulted,0.1425,0.1425,A
3,3,28073,68313,1199318400,100.0,Charge-off,0.3299,0.3299,C
4,4,9896,60769,1181606400,50.0,Paid,0.1411,0.1411,C


In [6]:
lender_table = data_df.Lender_ID.value_counts().to_frame().reset_index().rename(columns={"index": "lenderid", "Lender_ID": "freq"})

lender_table.head(20)

,lenderid,freq
0,290,9436
1,431,4631
2,416,4568
3,280,4347
4,349,4346
5,1314,4021
6,341,3981
7,291,3976
8,725,3959
9,3008,3744


In [7]:
lender_table.shape

(59046, 2)

In [8]:
borrower_table = data_df.Borrower_ID.value_counts().to_frame().reset_index().rename(columns={"index": "borrowerid", "Borrower_ID": "freq"})

borrower_table.head(20)

,borrowerid,freq
0,61617,1071
1,62310,1014
2,68833,1010
3,21330,921
4,64731,915
5,66507,886
6,63917,884
7,63897,882
8,21396,864
9,61963,863


In [9]:
borrower_table.shape

(35846, 2)

In [10]:
def convert_time(x):
    return datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')

In [11]:
data_df['Time'] = data_df.Timestamp.apply(lambda x: convert_time(x))

In [12]:
data_df.head()

,Unnamed: 0,Lender_ID,Borrower_ID,Timestamp,Amount,Status,Lender_Rate,Borrower_Rate,Grade,Time
0,0,20504,79275,1182816000,100.0,Charge-off,0.1750,0.1950,B,2007-06-26 00:00:00
1,1,6182,59872,1213142400,90.0,Paid,0.1355,0.1355,B,2008-06-11 00:00:00
2,2,10118,67702,1155600000,1000.0,Defaulted,0.1425,0.1425,A,2006-08-15 00:00:00
3,3,28073,68313,1199318400,100.0,Charge-off,0.3299,0.3299,C,2008-01-03 00:00:00
4,4,9896,60769,1181606400,50.0,Paid,0.1411,0.1411,C,2007-06-12 00:00:00


In [13]:
data_df = data_df.drop(['Unnamed: 0','Timestamp'],axis=1)

In [14]:
data_df.head()

,Lender_ID,Borrower_ID,Amount,Status,Lender_Rate,Borrower_Rate,Grade,Time
0,20504,79275,100.0,Charge-off,0.1750,0.1950,B,2007-06-26 00:00:00
1,6182,59872,90.0,Paid,0.1355,0.1355,B,2008-06-11 00:00:00
2,10118,67702,1000.0,Defaulted,0.1425,0.1425,A,2006-08-15 00:00:00
3,28073,68313,100.0,Charge-off,0.3299,0.3299,C,2008-01-03 00:00:00
4,9896,60769,50.0,Paid,0.1411,0.1411,C,2007-06-12 00:00:00


In [15]:
data_df['Quater'] = data_df.Time.apply(lambda x: pd.Timestamp(x).quarter)

In [16]:
data_df['Year'] = data_df.Time.apply(lambda x: pd.Timestamp(x).year)

In [17]:
data_df.head()

,Lender_ID,Borrower_ID,Amount,Status,Lender_Rate,Borrower_Rate,Grade,Time,Quater,Year
0,20504,79275,100.0,Charge-off,0.1750,0.1950,B,2007-06-26 00:00:00,2,2007
1,6182,59872,90.0,Paid,0.1355,0.1355,B,2008-06-11 00:00:00,2,2008
2,10118,67702,1000.0,Defaulted,0.1425,0.1425,A,2006-08-15 00:00:00,3,2006
3,28073,68313,100.0,Charge-off,0.3299,0.3299,C,2008-01-03 00:00:00,1,2008
4,9896,60769,50.0,Paid,0.1411,0.1411,C,2007-06-12 00:00:00,2,2007


In [18]:
h_list = ['Late','Defaulted','Current', '1', '2', '3', '4+', 'Charge-off', 'Cancelled']
l_list = ['Paid', 'Payoff', 'Repurchased']

In [19]:
#high risk = 0
#low risk = 1
def class_status(status, h_list, l_list):
    r = 99
    if status in h_list:
        r = 0
    elif status in l_list:
        r = 1
    else:
        print("missing")
    return r

In [20]:
data_df['loan_classification'] = data_df.Status.apply(lambda x: class_status(x,h_list,l_list))

In [21]:
data_df.head(30)

,Lender_ID,Borrower_ID,Amount,Status,Lender_Rate,Borrower_Rate,Grade,Time,Quater,Year,loan_classification
0,20504,79275,100.00,Charge-off,0.1750,0.1950,B,2007-06-26 00:00:00,2,2007,0
1,6182,59872,90.00,Paid,0.1355,0.1355,B,2008-06-11 00:00:00,2,2008,1
2,10118,67702,1000.00,Defaulted,0.1425,0.1425,A,2006-08-15 00:00:00,3,2006,0
3,28073,68313,100.00,Charge-off,0.3299,0.3299,C,2008-01-03 00:00:00,1,2008,0
4,9896,60769,50.00,Paid,0.1411,0.1411,C,2007-06-12 00:00:00,2,2007,1
5,11129,62480,100.00,Paid,0.1244,0.1244,A,2007-09-17 00:00:00,3,2007,1
6,24589,72456,50.00,Paid,0.2125,0.2125,C,2008-03-25 00:00:00,1,2008,1
7,52137,70310,150.00,Charge-off,0.1469,0.1469,C,2006-09-20 00:00:00,3,2006,0
8,28508,69388,100.00,Paid,0.1700,0.1700,D,2007-03-01 00:00:00,1,2007,1
9,2022,84015,100.00,Charge-off,0.1825,0.1900,A,2007-06-12 00:00:00,2,2007,0


In [22]:
data_df.loan_classification.value_counts()

0    1950906
1    1444073
Name: loan_classification, dtype: int64

In [23]:
(data_df.loan_classification == 0).sum()

1950906

In [24]:
data_df =data_df.sort_values(by=["Time"])

In [41]:
data_df['borrower_type'] = 99

In [33]:
#there is another quicker way to do it by sorting the bwr_ID
# for idx in data_df.index:
#     if data_df.Time[idx] == data_df.Time[idx+1]:
#         continue
#     else: 
#         temp_df = data_df.iloc[:idx,:]
#         new_rcds = temp_df[temp_df.Time == data_df.Time[idx]].index
#         for n_idx in new_rcds:
#             n_brw = temp_df.Borrower_ID[n_idx]
#             n_brw_rds = temp_df[temp_df.Borrower_ID == n_brw]
#             count_0 = (n_brw_rds.loan_classification == 0).sum()
#             count_1 = (n_brw_rds.loan_classification == 1).sum()
#             if count_0>= count_1:
#                 data_df.loc[idx,'borrower_type'] = -1
#             else:
#                 data_df.loc[idx,'borrower_type'] = 1 

In [31]:
bwr_list = list(set(data_df.Borrower_ID))

In [32]:
#quick method
records = {}
c = 0
for bwr in bwr_list:
    c += 1
    if c%1000 == 0:
        print(c)
    temp = data_df[data_df.Borrower_ID == bwr]
    for idx in temp.index:
        brw_rds = temp.loc[:idx,:]
        count_0 = (brw_rds.loan_classification == 0).sum()
        count_1 = (brw_rds.loan_classification == 1).sum()
        if count_0>= count_1:
            records[idx] = -1
        else:
            records[idx] = 1 

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


In [34]:
len(records)

3394979

In [30]:
data_df.shape

(3394979, 12)

In [35]:
records[0]

-1

In [44]:
borrower_type = pd.DataFrame.from_dict(records, orient = 'index')

In [49]:
data_df_fl = pd.merge(data_df,borrower_type,right_index=True, left_index =True, how = 'inner')

In [50]:
# c = 0
# for k, v in records.items():
#     c += 1
#     if c%1000 == 0:
#         print(c)
#     data_df.loc[k, 'borrower_type'] = v

In [52]:
data_df_fl.shape

(3394979, 13)

In [55]:
data_df_fl = data_df_fl.drop(['borrower_type'],axis=1)

In [57]:
data_df_fl = data_df_fl.rename(columns = {0: 'borrower_type'})

In [58]:
data_df_fl.head()

,Lender_ID,Borrower_ID,Amount,Status,Lender_Rate,Borrower_Rate,Grade,Time,Quater,Year,loan_classification,borrower_type
558701,267,4578,1000.0,Paid,0.04,0.04,AA,2005-11-15 00:00:00,4,2005,1,1
1153246,267,60169,1000.0,Paid,0.12,0.17,E,2005-11-18 00:00:00,4,2005,1,1
618543,267,1663,2500.0,Paid,0.25,0.25,HR,2005-11-22 00:00:00,4,2005,1,1
1204669,4037,75787,1000.0,Paid,0.13,0.13,AA,2005-11-22 00:00:00,4,2005,1,1
1290976,267,75787,1000.0,Paid,0.13,0.13,AA,2005-11-22 00:00:00,4,2005,1,1


In [61]:
data_df_fl.to_csv('data_yrqt_cl_bt.csv',index = False)

In [60]:
#check

In [69]:
data_df[data_df.Borrower_ID == 77799].shape

(230, 12)

In [71]:
tt = data_df_fl[data_df_fl.Borrower_ID == 61198]

In [74]:
tt

,Lender_ID,Borrower_ID,Amount,Status,Lender_Rate,Borrower_Rate,Grade,Time,Quater,Year,loan_classification,borrower_type
332165,46,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1071307,4997,61198,100.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1420356,5196,61198,100.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1504420,5004,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1064524,3769,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1934954,5217,61198,73.62,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
669138,65,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
948961,5036,61198,400.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1667360,5218,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
126072,21463,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1


In [77]:
tt.loc[:28,:]

,Lender_ID,Borrower_ID,Amount,Status,Lender_Rate,Borrower_Rate,Grade,Time,Quater,Year,loan_classification,borrower_type
332165,46,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1071307,4997,61198,100.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1420356,5196,61198,100.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1504420,5004,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1064524,3769,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1934954,5217,61198,73.62,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
669138,65,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
948961,5036,61198,400.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
1667360,5218,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1
126072,21463,61198,50.00,Paid,0.29,0.29,HR,2007-02-22 00:00:00,1,2007,1,1


In [76]:
tt.loc[:28,:].loan_classification.sum()

28

In [78]:
tt.loc[:28,:].shape

(67, 12)

### DONE

In [ ]:
test_2005 = data_df[(data_df.Quater == 4) & (data_df.Year == 2005)]

In [ ]:
test_2005

In [ ]:
data_df.Year.value_counts().to_frame().reset_index().rename(columns={'index': 'Year', 'Year':'num'})

In [ ]:
data_df_pivot = pd.pivot_table(data_df.loc[:,['Time','Year','Quater']], index = ['Year','Quater'], aggfunc='count')

In [ ]:
data_df_pivot

In [ ]:
data_df_pivot.index

In [ ]:
test = data_df[(data_df.Quater == 2) & (data_df.Year == 2006)]

In [ ]:
len(test)

In [ ]:
test.head()

In [ ]:
data_df.Status.unique()

In [ ]:
h_list = ['Late','Defaulted','Current', '1', '2', '3', '4+', 'Charge-off', 'Cancelled']
l_list = ['Paid', 'Payoff', 'Repurchased']

In [ ]:
#high risk = 1
#low risk = 0
def class_status(status, h_list, l_list):
    r = 99
    if status in h_list:
        r = 1
    elif status in l_list:
        r = 0
    else:
        print("missing")
    return r

In [ ]:
data_df['loan_classification'] = data_df.Status.apply(lambda x: class_status(x,h_list,l_list))

In [ ]:
data_df.head()

In [ ]:
#positive type = 1
#negative type = 0
def class_borrower_type(status_list,h_list,l_list):
    r = 99
    c_h = 0
    c_l = 0
    for s in status_list:
        if s in h_list:
            c_h += 1
        elif s in l_list:
            c_l += 1
        else:
            print("missing: ",s)
    if (c_h < c_l):
        r = 1
    else: r= 0
    return r

In [ ]:
borrowers_type = {}
for year_quater, fr in data_df.groupby(['Quater','Year']):
    print(year_quater)
    borrowers_type[year_quater] = [(0,0)]
    for bid, fr_2 in fr.groupby(['Borrower_ID']):
        status_list = list(fr_2.Status)
        b_type = class_borrower_type(status_list,h_list,l_list)
        borrower_type = (bid, b_type)
        borrowers_type[year_quater].append(borrower_type)

In [ ]:
borrowers_type

In [ ]:
with open ('borrowers_type_20200210','wb') as fp:
    pickle.dump(borrowers_type, fp)
fp.close()

In [ ]:
g_2006_3 = borrowers_type[(3,2006)][1:]

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
test_1 = test.iloc[0:100,:]

In [ ]:
len(test_1.Lender_ID.unique())

In [ ]:
len(test_1.Borrower_ID.unique())

In [ ]:
G_list = []
for idx in test_1.index:
    G_list.append([test_1.Lender_ID[idx],test_1.Borrower_ID[idx]])

In [ ]:
G_list

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
G_symmetric = nx.Graph()

In [ ]:
for g in G_list:
    G_symmetric.add_edge(g[1],g[0])

In [ ]:
print(nx.info(G_symmetric))

In [ ]:
plt.figure(figsize=(20,20))
nx.spring_layout(G_symmetric)
nx.draw_networkx(G_symmetric)

In [ ]:
test_1[test_1.Borrower_ID == 30055]

In [ ]:
M = nx.bfs_tree(G_symmetric, 30055, reverse=False)
plt.figure(figsize=(20,20))
nx.draw_networkx(M)

In [ ]:
M = nx.bfs_tree(G_symmetric, 71077, reverse=False)
plt.figure(figsize=(20,20))
nx.draw_networkx(M)

In [ ]:
plt.figure(figsize=(20,20))
nx.draw_networkx(G_symmetric);

In [ ]:
M = nx.bfs_tree(G_symmetric, 20807)
plt.figure(figsize=(20,20))
nx.draw_networkx(M)

In [ ]:
M = nx.bfs_tree(G_symmetric, 47410)
plt.figure(figsize=(20,20))
nx.draw_networkx(M)

In [ ]:
M = nx.bfs_tree(G_symmetric, 88154)
plt.figure(figsize=(20,20))
nx.spring_layout(M)
nx.draw_networkx(M)